In [1]:
import yfinance as yf
import pandas as pd
import requests
import plotly.graph_objects as go

In [2]:
def get_stock_data(ticker, start, end):
    stock = yf.Ticker(ticker)
    stock_data = stock.history(start=start, end=end)
    stock_data.reset_index(inplace=True)
    stock_data = stock_data[["Date", "Close"]]
    stock_data.rename(columns={"Close": "Stock Price"}, inplace=True)
    return stock_data

In [3]:
def get_revenue_data(ticker, start, end):
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{ticker.lower()}/revenue"
    headers = {"User-Agent": "Mozilla/5.0"}
    html = requests.get(url, headers=headers).text
    tables = pd.read_html(html)
    revenue = tables[1]
    revenue.columns = ["Date", "Revenue"]
    revenue.dropna(inplace=True)
    revenue = revenue[revenue["Revenue"] != "TTM"]
    revenue["Revenue"] = revenue["Revenue"].str.replace(r"[\$,]", "", regex=True)
    revenue = revenue[revenue["Revenue"] != ""]
    revenue["Revenue"] = revenue["Revenue"].astype(float)
    revenue["Date"] = pd.to_datetime(revenue["Date"])
    revenue = revenue.sort_values("Date", ascending=True).reset_index(drop=True)
    return revenue

In [4]:
def get_netprofit_data(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    net_income = financials.loc["Net Income"]
    revenue = financials.loc["Total Revenue"]
    netprofit = (net_income / revenue) * 100
    netprofit = netprofit.reset_index()
    netprofit.columns = ["Date", "Net Profit Margin %"]
    netprofit.dropna(inplace=True)
    netprofit["Date"] = pd.to_datetime(netprofit["Date"])
    netprofit = netprofit.sort_values("Date").reset_index(drop=True)
    return netprofit

In [5]:
def plot_graph(stock_data, revenue_data, ticker):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=stock_data["Date"],
            y=stock_data["Stock Price"],
            name="Stock Price",
            mode="lines",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=revenue_data["Date"],
            y=revenue_data["Revenue"],
            name="Revenue (Millions)",
            mode="lines",
            yaxis="y2",
        )
    )

    fig.update_layout(
        title=f"{ticker.upper()} Stock Price vs Revenue",
        xaxis_title="Date",
        yaxis_title="Stock Price",
        yaxis2=dict(title="Revenue (Millions)", overlaying="y", side="right"),
        legend=dict(x=0.1, y=0.9),
    )
    fig.show()

In [6]:
def plot_netprofit(netprofit, ticker):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=netprofit["Date"],
            y=netprofit["Net Profit Margin %"],
            mode="lines+markers",
            name="Net Profit Margin %",
        )
    )
    fig.update_layout(
        title=f"{ticker.upper()} Net Profit Margin Over Time",
        xaxis_title="Date",
        yaxis_title="Net Profit Margin %",
    )
    fig.show()

In [7]:
ticker = input("Enter a stock ticker symbol (e.g., TSLA, GME, AAPL): ").upper()
start = input("Enter start date (YYYY-MM-DD): ")
end = input("Enter end date (YYYY-MM-DD): ")
stock_data = get_stock_data(ticker, start=start, end=end)
revenue_data = get_revenue_data(ticker, start=start, end=end)
netprofit_data = get_netprofit_data(ticker)
revenue_data = revenue_data[
    (revenue_data["Date"] >= start) & (revenue_data["Date"] <= end)
]
plot_graph(stock_data, revenue_data, ticker)
plot_netprofit(netprofit_data, ticker)

ValueError: Empty ticker name